In [1]:
# import libs
import numpy as np
import pandas as pd
import os
import scipy.io as io

In [2]:
# def function

# 获取csv文件的列名
def getcols(filename):
    data = pd.read_csv(filename, nrows = 2)
    #print(data.columns)
    return data.columns

# 获取文件行数
def getrows(filename):
    cols = getcols(filename)
    col = cols[0]
    data = pd.read_csv(filename, usecols=[col])
    return data.shape[0]

# 数据预处理- step1: 删除无法用于计算的列与含空值、负值、inf值的行
def preprocess2017(data):
    labels = data[' Label']
    # 删除非数值型数据，这些数据暂时不用于聚类处理，但不代表这些数据没用
    dropset = ['Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
           ' Fwd Header Length.1', ' Label']
    # 删除含有大量负数的列
    drop_nega = [' Fwd Header Length', 'Init_Win_bytes_forward',
           ' Init_Win_bytes_backward', ' act_data_pkt_fwd',
       ' min_seg_size_forward']
    # 删除数据中可能出现的非数值型数据列
    drop_ifexist = ['External IP','SimillarHTTP', ' Inbound','Unnamed: 0', 'Unnamed: 0.1']
    
    # 如果有list中的columns则删除
    data.drop(data.columns[data.columns.isin(dropset+drop_nega+drop_ifexist)], axis = 1, inplace=True)
    
    data.dropna(how='any', inplace=True)
#    print(len(data))
    data.reset_index(inplace=True, drop=True)
    # 将所有数据类型转成float型
    data = data.astype('float')
    
    # 删除数据中inf的值 (需要连续执行两次，不知道原因，但连续执行后就可删除数据中的inf)
    idx_tuple = np.where(data.max(axis=1).values == np.inf)
    idx = list(idx_tuple[0])
   # print(idx)
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)
    labels.drop(axis=0, index = idx, inplace = True)
    labels.reset_index(inplace=True, drop=True)
   # print(len(data))
    idx_tuple = np.where(data.max(axis=1).values == np.inf)
    idx = list(idx_tuple[0])
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)
    labels.drop(axis=0, index = idx, inplace = True)
    labels.reset_index(inplace=True, drop=True)
   # print(len(data))

    # 删除数据中含负数的数据条数
    idx_tuple = np.where(data.min(axis=1).values < 0)
    idx = list(idx_tuple[0])
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)
    labels.drop(axis=0, index = idx, inplace = True)
    labels.reset_index(inplace=True, drop=True)
   # print(len(data))
    return data,labels

def preprocess2019(data, cols, labelnum, label):
    data.dropna(how='any', inplace=True)
    
    # print(data.columns)
    data.drop(cols[0:2],axis = 1, inplace = True)
    
    # 删除非数值型数据，这些数据暂时不用于聚类处理，但不代表这些数据没用
    dropset = ['Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
           ' Fwd Header Length.1', 'SimillarHTTP', ' Inbound']
    
    # 删除含有大量负数的列
    drop_nega = [' Fwd Header Length', 'Init_Win_bytes_forward',
           ' Init_Win_bytes_backward', ' act_data_pkt_fwd',
       ' min_seg_size_forward']
    data.drop(dropset, axis=1,inplace=True)
    data.drop(drop_nega, axis=1,inplace=True)
    
    # 将标签替换为数字
    data.replace(to_replace=label, value=labelnum, inplace=True)
    
    # 将所有数据类型转成float型
    data = data.astype('float')
    
    # 删除数据中inf的值
    idx_tuple = np.where(data.max(axis=1).values == np.inf)
    idx = list(idx_tuple[0])
#    print(len(idx))
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)
    
    idx_tuple = np.where(data.max(axis=1).values == np.inf)
    idx = list(idx_tuple[0])
#    print(len(idx))
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)
    
    # 删除数据中含负数的数据条数
    idx_tuple = np.where(data.min(axis=1).values < 0)
    idx = list(idx_tuple[0])
    data.drop(axis=0, index = idx, inplace = True)
    data.reset_index(inplace=True, drop=True)
    
    return data
    

In [3]:
# 从文件夹'/mnt/hgfs/linuxfile/victim2019/'中读取数据文件
filedir = '/mnt/hgfs/linuxfile/victim2019/'
files = os.listdir(filedir)

In [4]:
filedir2 = '/mnt/hgfs/linuxfile/2017/' 
benign_file = os.listdir(filedir2)[4]
benign_data = pd.read_csv(filedir2+benign_file)
tdata,labels = preprocess2017(benign_data)
benign = tdata.join(labels)

In [16]:
label = f[:-9]
print('starting to deal with ' + f)
allrows = getrows(filedir+f)
skiprow = allrows // (nrows + int(0.2 * nrows));
if skiprow == 0:
    print('data is too small in ' + f)
cols = getcols(filedir+f)
    
data = pd.read_csv(filedir+f, names=cols, skiprows=lambda x: (x==0 or (x % skiprow) > 0), nrows = nrows + int(0.2 * nrows))
tdata,labels = preprocess2017(data)
tdata = data.join(labels)
alldata = alldata.append(benign.loc[start:start+nrows-1])
alldata = alldata.append(tdata)
print(len(alldata))
start += nrows

starting to deal with DrDoS_LDAP_victim_data.csv
44000


/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (86) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
tdata

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Subflow Bwd Bytes,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_LDAP
1,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_LDAP
2,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_LDAP
3,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_LDAP
4,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_LDAP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
23996,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
23997,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
23998,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [29]:
nrows = 20000 # 每个文件中读取nrows行数据
start = 0
alldata = pd.DataFrame()
for f in files: # 循环读入数据
    label = f[:-9]
    print('starting to deal with ' + f)
    allrows = getrows(filedir+f)
    skiprow = allrows // (nrows + int(0.2 * nrows));
    if skiprow == 0:
        print('data is too small in ' + f)
        continue
    cols = getcols(filedir+f)
    
    data = pd.read_csv(filedir+f, names=cols, skiprows=lambda x: (x==0 or (x % skiprow) > 0), nrows = nrows + int(0.2 * nrows))
    tdata,labels = preprocess2017(data)
    tdata = data.join(labels)
    alldata = alldata.append(benign.loc[start:start+nrows-1])
    alldata = alldata.append(tdata)
#    print(len(alldata))
    start += nrows
    
alldata.reset_index(inplace=True, drop=True)
alldata.to_csv('/mnt/hgfs/linuxfile/2019riskdata.csv', mode='w+',index=False)
    

starting to deal with DrDoS_DNS_victim_data.csv


/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (86) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


starting to deal with DrDoS_LDAP_victim_data.csv


/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (86) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


starting to deal with DrDoS_MSSQL_victim_data.csv
starting to deal with DrDoS_NetBIOS_victim_data.csv
starting to deal with DrDoS_NTP_victim_data.csv


...

/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (86) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


starting to deal with DrDoS_SSDP_victim_data.csv
starting to deal with DrDoS_UDP_victim_data.csv


/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (86) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


starting to deal with Syn_victim_data.csv
starting to deal with TFTP_victim_data.csv
starting to deal with UDPLag_victim_data.csv


/home/xuhao/miniconda3/envs/grdenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (86) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
# nrows = 20000 # 每个文件中读取nrows行数据
# start = 0
# for f in files: # 循环读入数据
#     label = f[:-9]
#     print('starting to deal with ' + f)
#     allrows = getrows(filedir+f)
#     skiprow = allrows // (nrows + int(0.2 * nrows));
#     if skiprow == 0:
#         print('data is too small in ' + f)
#         continue
#     cols = getcols(filedir+f)
    
#     tdata = pd.read_csv(filedir+f, names=cols, skiprows=lambda x: (x==0 or (x % skiprow) > 0), nrows = nrows + int(0.2 * nrows))
#     data,labels = preprocess2017(tdata)
#     alldata = data.join(labels)
#     if alldata.empty == False:
#         if os.path.exists(filedir + 'riskdata.csv') == False:
#             alldata.to_csv(filedir +'riskdata.csv', mode='a+')
#         else:
#             alldata.to_csv(filedir +'riskdata.csv', mode='a+', header = False)
    

In [31]:
alldata

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Subflow Bwd Bytes,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,166.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,BENIGN
1,83.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,BENIGN
2,99947.0,1.0,1.0,48.0,48.0,48.0,48.0,48.0,0.0,48.0,...,48.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,BENIGN
3,37017.0,1.0,1.0,48.0,48.0,48.0,48.0,48.0,0.0,48.0,...,48.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,BENIGN
4,111161336.0,147.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1753752.625,2123197.578,4822992.0,95.0,9463032.7,2657727.996,13600000.0,5700287.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478575,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,UDP-lag
478576,49.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,UDP-lag
478577,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,UDP-lag
478578,49.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,UDP-lag


In [32]:
data = pd.read_csv('/mnt/hgfs/linuxfile/2019riskdata.csv')

In [33]:
data

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Subflow Bwd Bytes,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,166.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,BENIGN
1,83.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,BENIGN
2,99947.0,1.0,1.0,48.0,48.0,48.0,48.0,48.0,0.0,48.0,...,48.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,BENIGN
3,37017.0,1.0,1.0,48.0,48.0,48.0,48.0,48.0,0.0,48.0,...,48.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,BENIGN
4,111161336.0,147.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1753752.625,2123197.578,4822992.0,95.0,9463032.7,2657727.996,13600000.0,5700287.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478575,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,UDP-lag
478576,49.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,UDP-lag
478577,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,UDP-lag
478578,49.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0,UDP-lag


In [28]:
help(pd.DataFrame.to_csv)

Help on function to_csv in module pandas.core.generic:

to_csv(self, path_or_buf: 'Optional[FilePathOrBuffer]' = None, sep: 'str' = ',', na_rep: 'str' = '', float_format: 'Optional[str]' = None, columns: 'Optional[Sequence[Label]]' = None, header: 'Union[bool_t, List[str]]' = True, index: 'bool_t' = True, index_label: 'Optional[IndexLabel]' = None, mode: 'str' = 'w', encoding: 'Optional[str]' = None, compression: 'CompressionOptions' = 'infer', quoting: 'Optional[int]' = None, quotechar: 'str' = '"', line_terminator: 'Optional[str]' = None, chunksize: 'Optional[int]' = None, date_format: 'Optional[str]' = None, doublequote: 'bool_t' = True, escapechar: 'Optional[str]' = None, decimal: 'str' = '.', errors: 'str' = 'strict', storage_options: 'StorageOptions' = None) -> 'Optional[str]'
    Write object to a comma-separated values (csv) file.
    
    .. versionchanged:: 0.24.0
        The order of arguments for Series was changed.
    
    Parameters
    ----------
    path_or_buf : str o